In [ ]:
%pip install git+https://github.com/fptprdqs66-dot/nytbee_scrapper.git
from nytbee_solver import get_default_wordlist_path, get_todays_puzzle_letters, print_hint_page, solve_spelling_bee


In [ ]:
try:
    default_letters = get_todays_puzzle_letters()
except Exception as exc:
    print(f"Unable to fetch today's letters: {exc}")
    default_letters = ""

if default_letters:
    prompt = (
        "Enter the Spelling Bee letters (required letter first) "
        f"[default: {default_letters}]: "
    )
else:
    prompt = "Enter the Spelling Bee letters (required letter first): "

letters_input = input(prompt).strip() or default_letters
words, pangrams, cleaned_letters, required_letter = solve_spelling_bee(
    letters_input, get_default_wordlist_path()
)
print_hint_page(words, pangrams, cleaned_letters, required_letter)
